In [ ]:
pip install langchain langchain-text-splitters langchain-community bs4

                                              0.0/111.7 kB ? eta -:--:--
                                              0.0/111.7 kB ? eta -:--:--
     ---                                      10.2/111.7 kB ? eta -:--:--
     ----------                            30.7/111.7 kB 330.3 kB/s eta 0:00:01
     ------------------------------------ 111.7/111.7 kB 725.0 kB/s eta 0:00:00
                                              0.0/2.5 MB ? eta -:--:--
     --                                       0.2/2.5 MB 3.5 MB/s eta 0:00:01
     ----                                     0.3/2.5 MB 3.5 MB/s eta 0:00:01
     --------------                           0.9/2.5 MB 6.4 MB/s eta 0:00:01
     -----------------------                  1.5/2.5 MB 8.0 MB/s eta 0:00:01
     ---------------------------------------  2.5/2.5 MB 10.6 MB/s eta 0:00:01
     ---------------------------------------- 2.5/2.5 MB 10.1 MB/s eta 0:00:00
                                              0.0/501.4 kB ? eta -:--:--
     -

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-client 1.12.3 requires platformdirs<5.0,>=3.10.0, but you have platformdirs 3.0.0 which is incompatible.
conda 24.7.1 requires platformdirs>=3.10.0, but you have platformdirs 3.0.0 which is incompatible.
conda-repo-cli 1.0.88 requires platformdirs>=3.10.0, but you have platformdirs 3.0.0 which is incompatible.


In [ ]:
import getpass
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "lsv2_xxxx"

In [3]:
pip install -U langchain-openai

                                              0.0/87.2 kB ? eta -:--:--
     ----                                     10.2/87.2 kB ? eta -:--:--
     ----                                     10.2/87.2 kB ? eta -:--:--
     -----------------                      41.0/87.2 kB 330.3 kB/s eta 0:00:01
     -----------------------------------    81.9/87.2 kB 512.0 kB/s eta 0:00:01
     -------------------------------------- 87.2/87.2 kB 493.4 kB/s eta 0:00:00
                                              0.0/1.1 MB ? eta -:--:--
     --                                       0.1/1.1 MB 3.2 MB/s eta 0:00:01
     --------------------                     0.6/1.1 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------  1.1/1.1 MB 9.9 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 9.9 MB/s eta 0:00:00
                                              0.0/879.4 kB ? eta -:--:--
     ------------------------------------- 879.4/879.4 kB 18.5 MB/s eta 0:00:00
  Usin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 24.7.1 requires platformdirs>=3.10.0, but you have platformdirs 3.0.0 which is incompatible.


In [10]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [11]:
print("Key loaded:", bool(os.environ.get("OPENAI_API_KEY")))

Key loaded: True


In [27]:
import os
from langchain.chat_models import init_chat_model

# os.environ["OPENAI_API_KEY"] = "sk-..."

model = init_chat_model("gpt-4.1")

In [16]:
pip install -U "langchain-core"

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)


In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047


In [20]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


In [6]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
##Building an agent to evaluate retrieval context which is the tool is necessary or not
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [30]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_s1LfhZNaKVcQjNSPx9o6Rw6p)
 Call ID: call_s1LfhZNaKVcQjNSPx9o6Rw6p
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external cla

In [29]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_3IJIo1WMa5EQ6on0QsRBh34I)
 Call ID: call_3IJIo1WMa5EQ6on0QsRBh34I
  Args:
    query: task decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL

In [12]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [38]:
pip install faiss-cpu

                                              0.0/18.9 MB ? eta -:--:--
                                              0.0/18.9 MB ? eta -:--:--
                                              0.0/18.9 MB ? eta -:--:--
                                             0.1/18.9 MB 544.7 kB/s eta 0:00:35
                                              0.3/18.9 MB 2.1 MB/s eta 0:00:09
     --                                       1.1/18.9 MB 6.6 MB/s eta 0:00:03
     ----                                     2.2/18.9 MB 9.9 MB/s eta 0:00:02
     ------                                   3.2/18.9 MB 12.1 MB/s eta 0:00:02
     ---------                                4.3/18.9 MB 13.7 MB/s eta 0:00:02
     ----------                               5.2/18.9 MB 14.3 MB/s eta 0:00:01
     -------------                            6.3/18.9 MB 15.3 MB/s eta 0:00:01
     ---------------                          7.3/18.9 MB 16.1 MB/s eta 0:00:01
     -----------------                        8.4/18.9 MB 16.8 MB/s

In [13]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [14]:
retriever = vectorstore.as_retriever()

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini",   # safe default
    temperature=0
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
 ##Building a RAG chain without an agent
prompt = ChatPromptTemplate.from_template("""
Answer using the context below.

Context:
{context}

Question:
{question}
""")

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

chain = (
    {
        "context": retriever | format_docs,
        "question": lambda x: x
    }
    | prompt
    | model
)

chain.invoke("What is task decomposition?")

AIMessage(content='Task decomposition is the process of breaking down a complicated or large task into smaller, simpler, and more manageable sub-tasks or steps. This helps an agent or model plan ahead and tackle each part step-by-step, making it easier to solve complex problems.\n\nIn the context provided, task decomposition is often achieved using techniques like Chain of Thought (CoT), where the model is prompted to "think step by step" to divide a big task into multiple smaller tasks. Another advanced method is Tree of Thoughts, which explores multiple reasoning possibilities at each step, creating a tree structure of thoughts to find the best solution.\n\nTask decomposition can be done in several ways:\n1. By a large language model (LLM) with simple prompting, such as asking for the steps or subgoals to achieve a task.\n2. Using task-specific instructions tailored to the domain, like writing a story outline for a novel.\n3. With human input guiding the breakdown.\n\nThere is also a